# LSTM for Sentiment Analysis

Following [this tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)

Helpful blogs:
* [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)